In [185]:
%%bash --out TOKEN
# Step 1: Authenticate and extract nonce using jq
nonce=$(curl -s 'http://localhost:7070/auth/authenticate' \
  -H 'Accept: application/json, text/plain, */*' \
  -H 'Accept-Language: en-GB,en-US;q=0.9,en;q=0.8' \
  -H 'Connection: keep-alive' \
  -H 'Content-Type: application/json' \
  -H 'Origin: http://localhost:3020' \
  -H 'Referer: http://localhost:3020/' \
  -H 'Sec-Fetch-Dest: empty' \
  -H 'Sec-Fetch-Mode: cors' \
  -H 'Sec-Fetch-Site: same-site' \
  -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36' \
  -H 'sec-ch-ua: "Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "macOS"' \
  --data-raw '{"username":"k.mweene","password":"test"}' | jq -r '.nonce')

# Step 2: Use the extracted nonce in the verifyCode request
TOKEN=$(curl -s 'http://localhost:7070/auth/verifyCode' \
  -H 'Accept: application/json, text/plain, */*' \
  -H 'Accept-Language: en-GB,en-US;q=0.9,en;q=0.8' \
  -H 'Connection: keep-alive' \
  -H 'Content-Type: application/json' \
  -H 'Origin: http://localhost:3020' \
  -H 'Referer: http://localhost:3020/' \
  -H 'Sec-Fetch-Dest: empty' \
  -H 'Sec-Fetch-Mode: cors' \
  -H 'Sec-Fetch-Site: same-site' \
  -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36' \
  -H 'sec-ch-ua: "Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "macOS"' \
  --data-raw "{\"code\":\"000000\",\"nonce\":\"$nonce\"}" | jq -r '.token')

echo "$TOKEN"

# Create an "event"

In [186]:
%%bash -s "$TOKEN" --out id

TOKEN=$1
response_data=$(curl --request POST \
  --url "http://localhost:7070/graphql" \
  --header 'Authorization: Bearer '$TOKEN \
  --header 'Content-Type: application/json' \
  -d '{
      "query": "mutation CreateEvent($input: EventInput!) { createEvent(event: $input) { id type createdAt updatedAt actions { ... on CreateAction { type createdAt createdBy data { id value } } } } }",
      "variables": { "input": { "type": "tennis-club-membership" } }
    }')

id=$(echo "$response_data" | jq '.data.createEvent.id')
echo "$id"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   577  100   299  100   278    686    638 --:--:-- --:--:-- --:--:--  1323


In [187]:
%%bash -s "$id" "$TOKEN"

id=$(echo "$1" | xargs)
TOKEN=$2

echo "Event ID: '$id'"
echo "Token: $TOKEN"

response_data=$(curl --request POST \
  --url "http://localhost:7070/graphql" \
  --header 'Authorization: Bearer '$TOKEN \
  --header 'Content-Type: application/json' \
  -d '{
      "query": "mutation DeclareEvent($eventId: ID!, $input: DeclareActionInput!) { declareEvent(eventId: $eventId, input: $input) { id type createdAt updatedAt actions { ... on DeclareAction { type createdAt createdBy data { id value } } } } }",
      "variables": {
        "eventId": "'${id}'",
        "input": {
          "data": [
            { "id": "applicant.firstname", "value": "Riku" },
            { "id": "applicant.surname", "value": "Rouvila" },
            { "id": "applicant.dob", "value": "2024-11-27T11:56:20.781Z" }
          ]
        }
      }
    }')
echo "$response_data" | jq

Event ID: '1e2404d8-0c90-4a34-91c7-fe1f27899b38'
Token: eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJzY29wZSI6WyJyZWdpc3RlciIsInBlcmZvcm1hbmNlIiwiY2VydGlmeSIsImRlbW8iXSwiaWF0IjoxNzMyNzA5MTk0LCJleHAiOjE3MzMzMTM5OTQsImF1ZCI6WyJvcGVuY3J2czphdXRoLXVzZXIiLCJvcGVuY3J2czp1c2VyLW1nbnQtdXNlciIsIm9wZW5jcnZzOmhlYXJ0aC11c2VyIiwib3BlbmNydnM6Z2F0ZXdheS11c2VyIiwib3BlbmNydnM6bm90aWZpY2F0aW9uLXVzZXIiLCJvcGVuY3J2czp3b3JrZmxvdy11c2VyIiwib3BlbmNydnM6c2VhcmNoLXVzZXIiLCJvcGVuY3J2czptZXRyaWNzLXVzZXIiLCJvcGVuY3J2czpjb3VudHJ5Y29uZmlnLXVzZXIiLCJvcGVuY3J2czp3ZWJob29rcy11c2VyIiwib3BlbmNydnM6Y29uZmlnLXVzZXIiLCJvcGVuY3J2czpkb2N1bWVudHMtdXNlciJdLCJpc3MiOiJvcGVuY3J2czphdXRoLXNlcnZpY2UiLCJzdWIiOiI2NzMzMzA5ODI3Yjk3ZTY0ODM4NzcxODgifQ.RN47J3mCQjC1ODxdFCRDdA8MURkYvwtbdLURpwRphvi8-MpjFr5oO_38CGLyqsyrR2uLn8R1IBmhxieN1zxqJJNYqTZHhLviAFQuL9wTnYVcNw40xA6A69y-AeEstdLOgzB5kBYBJIBPW83DCpIizmSd25eIvk-Wv8zh69ybDXZ7uFJ3eiOW-kBjpT6SGj9ZX27IEODLrsrSIdM098dz-QQpwiywrjm5Oz2S9VQGmgU1ORQTrcyJDwK1mB7_SuNhbF3i1FHqd5hienYpp388OekhzjisN0M_TbWeeNwm

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1053  100   450  100   603  22114  29632 --:--:-- --:--:-- --:--:-- 52650


In [193]:
%%bash -s "$id" "$TOKEN"

id=$(echo "$1" | xargs)
TOKEN=$2


response_data=$(curl --request POST \
  --url "http://localhost:7070/graphql" \
  --silent \
  --header 'Authorization: Bearer '$TOKEN \
  --header 'Content-Type: application/json' \
  -d '{
      "query": "query GetEvent($eventId: ID!) { getEvent(eventId: $eventId) { id type createdAt updatedAt actions { ... on CreateAction { type createdAt createdBy data { id value } } ... on NotifyAction { type createdAt createdBy data { id value } } ... on DeclareAction { type createdAt createdBy data { id value } } ... on RegisterAction { type createdAt createdBy data { id value } } } } }",
      "variables": {
        "eventId": "'${id}'"
      }
    }')
echo "$response_data" | jq

{
  "data": {
    "getEvent": {
      "id": "1e2404d8-0c90-4a34-91c7-fe1f27899b38",
      "type": "tennis-club-membership",
      "createdAt": "2024-11-27T12:06:34.941Z",
      "updatedAt": "2024-11-27T12:06:34.941Z",
      "actions": [
        {
          "type": "CREATE",
          "createdAt": "2024-11-27T12:06:34.941Z",
          "createdBy": "6733309827b97e6483877188",
          "data": []
        },
        {
          "type": "DECLARE",
          "createdAt": "2024-11-27T12:06:37.275Z",
          "createdBy": "6733309827b97e6483877188",
          "data": [
            {
              "id": "applicant.firstname",
              "value": "Riku"
            },
            {
              "id": "applicant.surname",
              "value": "Rouvila"
            },
            {
              "id": "applicant.dob",
              "value": "2024-11-27T11:56:20.781Z"
            }
          ]
        }
      ]
    }
  }
}
